In [ ]:
import pandas as pd

from magine.enrichment.enrichment_result import EnrichmentResult
from magine.data.experimental_data import ExperimentalData

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.family'] = ['arial']
# matplotlib.rcParams['font.size'] = 6

sns.set_theme(
    context ='paper',
    palette="Paired", 
    style='white',
    font='arial',
    # font_scale=1.0
)

In [ ]:
exp_data = ExperimentalData('../data/global_proteomics_for_magine.csv.gz')
both_exp_data = ExperimentalData('../data/both_proteomics_for_magine.csv.gz')

In [ ]:
from magine.enrichment.enrichment_result import load_enrichment_csv

In [ ]:
enr_results_down = load_enrichment_csv('../Data/ora_results_down.csv')
enr_results_up = load_enrichment_csv('../Data/ora_results_up.csv')
enr_results_both = load_enrichment_csv('../Data/ora_results_both.csv')

In [ ]:
def mod_df(df):
    df['stage'] = df.sample_id.str.split('_').str.get(1)
    df['drug'] = df.sample_id.str.split('_').str.get(0)
    df.loc[df.stage=='only', 'stage'] = 'Dec only'
    df.loc[df.drug=='D', 'drug'] = '0D'
    df.loc[df.drug=='GD', 'drug'] = '1GD'
    df.loc[df.drug=='GVD', 'drug'] = '2GVD'
    df.loc[df.drug=='GV', 'drug'] = '3GV'
    df.loc[df.drug=='G', 'drug'] = '4G'
mod_df(enr_results_down)
mod_df(enr_results_up)
mod_df(enr_results_both)

In [ ]:
# make plotting easier for ranking    
exp = exp_data.species
mod_df(exp)
exp_both = both_exp_data.species
mod_df(exp_both)

In [ ]:
reactome_results = enr_results_up.filter_multi(db='Reactome_2022')
reactome_results['term_name'] = reactome_results.term_name.str.split(' r-hsa').str.get(0)

In [ ]:
reactome_results

In [ ]:
reactome_results.remove_redundant(level='dataframe'
).heatmap(
    figsize=(10, 10),
    cluster_row=False,
    columns=['drug', 'stage'],
    y_tick_labels=True,
    linewidths=0.01
);
plt.savefig('reactome_changes_heatmap.png', dpi=300, bbox_inches='tight')

In [ ]:
cellmarker_results_dn =  enr_results_down.filter_multi(db='CellMarker_2024')
cellmarker_results_dn = cellmarker_results_dn.loc[(cellmarker_results_dn.term_name.str.contains('bone') | 
                                                  cellmarker_results_dn.term_name.str.contains('blood'))&
                                                  cellmarker_results_dn.term_name.str.contains('human')
]

cellmarker_results_up = enr_results_up.filter_multi(db='CellMarker_2024')
cellmarker_results_up = cellmarker_results_up.loc[(cellmarker_results_up.term_name.str.contains('bone') | 
                                                  cellmarker_results_up.term_name.str.contains('blood')) &
                                                  cellmarker_results_up.term_name.str.contains('human')]

In [ ]:
hubmap_results_dn =  enr_results_down.filter_multi(db='HuBMAP_ASCTplusB_augmented_2022')
hubmap_results_dn = hubmap_results_dn.loc[(hubmap_results_dn.term_name.str.contains('bone') | 
                                           hubmap_results_dn.term_name.str.contains('blood'))
]

hubmap_results_up = enr_results_up.filter_multi(db='HuBMAP_ASCTplusB_augmented_2022')
hubmap_results_up = hubmap_results_up.loc[(hubmap_results_up.term_name.str.contains('bone') |
                                           hubmap_results_up.term_name.str.contains('blood'))
]
hubmap_results_up

In [ ]:
def gather_susbets(samples, df, savename='test', min_sig=1, figsize=(10, 8)):
    
    desired_samples = df.sig.copy()
    desired_samples = desired_samples.loc[desired_samples.sample_id.isin(samples)]
    desired_samples = set(desired_samples.term_name)
    
    other_samples = df.sig.copy()
    other_samples = other_samples.loc[~(other_samples.sample_id.isin(samples))]
    
    other_samples = set(other_samples.sig.term_name)
    interesting_terms = list(desired_samples.difference(other_samples))
                             
    subset = df.filter_rows('term_name', interesting_terms)
    if len(subset) == 0:
        print('No terms found')
        return
    # min_sig = 2 if len(samples) > 3 else 1
    subset.require_n_sig(n_sig=min_sig, index='term_name').remove_redundant(
        level='dataframe').heatmap(
        figsize=figsize,
        cluster_row=True,
        columns=['drug', 'stage'],
        # cluster_by_set=True,
        # min_sig=min_sig,
        y_tick_labels=True,
        linewidths=0.01
    );
    plt.savefig(f"heatmap_{savename}.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"heatmap_{savename}.pdf", dpi=300, bbox_inches='tight')

def plot_gene_list(gs, figsize=(8,8), savename='tmp', index='identifier'):
    
    exp.subset(gs, index=index).require_n_sig(n_sig=1, index='label').heatmap(
        columns=['drug', 'stage'],
        index='label',
        figsize=figsize,
        cluster_col=False,
        cluster_row=True,
        cluster_by_set=False,
        linewidths=0.01,
        y_tick_labels=True
    );
    # plt.suptitle(term)
    plt.savefig(f"{savename}_only_signals.png".replace(' ', '_').replace('/', '_'), dpi=200, bbox_inches='tight')
    plt.savefig(f"{savename}_only_signals.pdf".replace(' ', '_').replace('/', '_'), dpi=200, bbox_inches='tight')

def plot_subset(enrichment, term, figsize=(5,5), prefix='', y_tick_labels=True):
    gs = enrichment.term_to_genes(term)
    exp.subset(gs).require_n_sig(n_sig=1, index='label').heatmap(
        columns=['drug', 'stage'],
        index='label',
        figsize=figsize,
        cluster_col=False,
        cluster_row=True,
        cluster_by_set=False,
        linewidths=0.01,
        y_tick_labels=y_tick_labels
    );
    # plt.suptitle(term)
    plt.savefig(f"{term}_only_signals_{prefix}.png".replace(' ', '_').replace('/', '_'), dpi=200, bbox_inches='tight')
    plt.savefig(f"{term}_only_signals_{prefix}.pdf".replace(' ', '_').replace('/', '_'), dpi=200, bbox_inches='tight')
    

In [ ]:
no_dec_list = ['G_Early', 'G_Late', 'GV_Early', 'GV_Late']
venetoclax_list = ['GVD_Early', 'GVD_Late', 'GV_Early', 'GV_Late']
dec_list = ['D_only', 'GD_Early', 'GD_Late', 'GVD_Early', 'GVD_Late']
all_samples = set(enr_results_up.sample_id.unique())

In [ ]:
gather_susbets(dec_list, cellmarker_results_up, min_sig=2, figsize=(6,6), savename='cellmarker_decitabine_any_up')

In [ ]:
cellmarker_results_up.term_name.unique()

In [ ]:
plot_subset(cellmarker_results_up, 'neutrophil blood human')

In [ ]:

plot_subset(cellmarker_results_up, 'hematopoietic stem cell bone marrow human')

In [ ]:
gather_susbets(venetoclax_list, reactome_results, min_sig=1, figsize=(4,4), savename='reatome_vent_list')

In [ ]:
gather_susbets(dec_list, hubmap_results_up, min_sig=2, figsize=(6,6), savename='hubmap_up')

In [ ]:
hubmap_results_up.require_n_sig(n_sig=2, index='term_name').remove_redundant(threshold=.5, level='dataframe').heatmap(
    figsize=(6, 6),
    cluster_row=False,
    columns=['drug', 'stage'],
    y_tick_labels=True,
    linewidths=0.01
);
plt.savefig('hubmap_up.png', dpi=300, bbox_inches='tight')
plt.savefig('hubmap_up.pdf', dpi=300, bbox_inches='tight')

In [ ]:
hubmap_results_dn.require_n_sig(n_sig=1, index='term_name').remove_redundant().heatmap(
    figsize=(10, 10),
    cluster_row=True,
    columns=['drug', 'stage'],
    y_tick_labels=True,
    linewidths=0.01
);

In [ ]:
plot_subset(hubmap_results_up, 'promonocyte - bone marrow', prefix='up')
plot_subset(hubmap_results_up, 'cd14-positive monocyte - bone marrow', prefix='up')

In [ ]:
term = 'megakaryocyte-erythroid progenitor cell - bone marrow'
megakaryocyte = hubmap_results_up.term_to_genes(term)
megakaryocyte.update(hubmap_results_dn.term_to_genes(term))
plot_gene_list(megakaryocyte, figsize=(4, 6), savename='megakaryocyte_erythroid_progenitor_cell')

In [ ]:
term = 'promonocyte - bone marrow'
pro_mon = hubmap_results_up.term_to_genes(term)
pro_mon.update(hubmap_results_dn.term_to_genes(term))
plot_gene_list(pro_mon, figsize=(6, 8), savename='promonocyte')

In [ ]:
term = 'cd14-positive monocyte - bone marrow'
cd14_pos = hubmap_results_up.term_to_genes(term)
cd14_pos.union(hubmap_results_dn.term_to_genes(term))
plot_gene_list(cd14_pos, figsize=(4, 10), savename='cd14_positive')

In [ ]:
plot_subset(hubmap_results_dn, 'promonocyte - bone marrow')
plot_subset(hubmap_results_dn, 'cd14-positive monocyte - bone marrow')

In [ ]:
plot_subset(hubmap_results_dn, 'promonocyte - bone marrow')

In [ ]:
plot_subset(hubmap_results_dn, 'cd14-low, cd16-positive monocyte - bone marrow')

In [ ]:

gather_susbets(
    dec_list,
    df=reactome_results,
    savename='reactome_decitabine_any',
    min_sig=2,
    figsize=(16, 12)
)

In [ ]:

gather_susbets(
    no_dec_list,
    df=reactome_results,
    savename='reactome_no_decitabine',
    min_sig=1,
    figsize=(8, 4)
)

In [ ]:
gather_susbets(
    venetoclax_list, 
    cellmarker_results_up, 
    min_sig=1, figsize=(8, 8),
    savename='cellmarker_ventoclax_any',
)

In [ ]:

gather_susbets(
    dec_list,
    df=cellmarker_results_up,
     figsize=(14, 8),
    savename='cellmarker_decitabine_any',
    min_sig=2,
)
gather_susbets(
    dec_list,
    df=cellmarker_results_dn,
     figsize=(18, 8),
    savename='cellmarker_decitabine_dn',
    min_sig=1,
)

In [ ]:
gather_susbets(
    ['D_only',],# 'GD_Early', 'GD_Late', 'GVD_Early', 'GVD_Late'],
    df=enr_results_down,
    # savename='cellmarker_decitabine_any',
    min_sig=1,
    figsize=(16,16)
)

In [ ]:

chea = enr_results_up.filter_multi(db='ChEA_2022').filter_based_on_words('human')
chea['term_name'] = chea.term_name.str.split(' ').str.get(0)

In [ ]:
chea.sig

In [ ]:

gather_susbets(
    dec_list,
    df=chea,
    savename='chea_both',
    min_sig=1,
    figsize=(5,10)
)

In [ ]:

trrust_up = enr_results_up.filter_multi(db='TRRUST_Transcription_Factors_2019').filter_based_on_words('human')
trrust_dn = enr_results_down.filter_multi(db='TRRUST_Transcription_Factors_2019').filter_based_on_words('human')

gather_susbets(
    dec_list,
    df=trrust_up,
    savename='tf_up_decitabine_any',
    min_sig=1,
    figsize=(4,8)
)

gather_susbets(
    dec_list,
    df=trrust_dn,
    savename='tf_dn_decitabine_any',
    min_sig=1,
    figsize=(3,6)
)


In [ ]:
plot_subset(trrust_dn, 'men1 human')

In [ ]:
plot_subset(trrust_up, 'cebpa human')

In [ ]:
plot_subset(trrust_dn, 'stat3 human')

In [ ]:
plot_subset(trrust_up, 'cebpa human')

In [ ]:
plot_subset(trrust_up, 'runx1 human')

In [ ]:
plot_subset(trrust_up, 'hivep2 human')

In [ ]:
encode_up = enr_results_up.filter_multi(db='ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X')
encode_dn = enr_results_down.filter_multi(db='ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X')
encode_both = enr_results_both.filter_multi(db='ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X')
encode = EnrichmentResult(pd.concat([encode_up, encode_dn]))

In [ ]:
gather_susbets(
    dec_list,
    df=encode_both,
    savename='tf_encode_decitabine_any',
    min_sig=1,
    figsize=(4,8)
)

In [ ]:
gather_susbets(
    dec_list,
    df=encode_up,
    savename='tf_encode_decitabine_up',
    min_sig=3,
    figsize=(4,5)
)

In [ ]:
encode_dn.sort_values('z_score', ascending=True)

In [ ]:
gather_susbets(
    dec_list,
    df=encode_dn,
    savename='tf_encode_decitabine_dn',
    min_sig=1,
    figsize=(4,8)
)

In [ ]:
sig_tfs = encode_dn.sig.term_name.str.upper().str.split(' ').str.get(0).to_list()
sig_tfs += set(encode_up.sig.term_name.str.upper().str.split(' ').str.get(0).to_list())
# sig_tfs += set(encode_both.sig.term_name.str.upper().str.split(' ').str.get(0).to_list())

In [ ]:
sig_tfs = set(sorted(sig_tfs))
sig_tfs

In [ ]:
# plot_gene_list(sig_tfs, figsize=(6, 4), savename='chea_tfs')

exp.subset(sig_tfs, index='identifier').require_n_sig(n_sig=1, index='label').heatmap(
        columns=['drug', 'stage'],
        index='label',
        figsize=(3,4),
        cluster_col=False,
        cluster_row=True,
        cluster_by_set=False,
        linewidths=0.01,
        y_tick_labels=True
    );
plt.savefig("transciption_factors.png", dpi=300, bbox_inches='tight')
plt.savefig("transciption_factors.pdf", dpi=300, bbox_inches='tight')

In [ ]:
term = 'gata2 chea'
plot_subset(encode, term, figsize=(4, 10), y_tick_labels=False)

In [ ]:
term = 'spi1 chea'
plot_subset(encode, term, figsize=(6, 8), y_tick_labels=False)

In [ ]:
term = 'myc chea'
plot_subset(encode, term, figsize=(4, 6), y_tick_labels=False)

In [ ]:
term = 'stat3 encode'
plot_subset(encode, term, figsize=(6, 8))

In [ ]:

gather_susbets(
    dec_list,
    df=encode_up,
    # savename='cellmarker_decitabine_any',
    min_sig=2,
    figsize=(8,12)
)

gather_susbets(
    dec_list,
    df=encode_dn,
    # savename='cellmarker_decitabine_any',
    min_sig=1,
    figsize=(8,12)
)

In [ ]:
gns = [ 'FLT3', 'STAT5A',  'PTPN11', 'STAT5B', 'SOCS2', 'CDKN1B', 'BBC3', 'BCL2L1', 'BCL2L11', 'SPI1', 'MYC', 'GATA1', 'GATA2', 'CEBPE']

# gns = [ 'FLT3', 'STAT5A',  'PTPN11', 'STAT5B', 'SOCS2', 'CDKN1B', 'BBC3', 'BCL2L1', 'SPI1']
# gns += enr_results_down.term_to_genes('stat5 activation downstream of flt3 itd mutants r-hsa-9702518')
exp.subset(gns, index='identifier',).require_n_sig(n_sig=1, index='identifier').heatmap(
    subset_index='identifier',
#     min_sig=1,
    index='label',
    columns=['drug', 'stage'],
    figsize=(4, 4), cluster_col=False,
    cluster_row=True,
    linewidths=0.01,
    y_tick_labels=True
);
plt.savefig("downstream_flt32.png", dpi=300, bbox_inches='tight')
plt.savefig("downstream_flt32.pdf", dpi=300, bbox_inches='tight')

## Plotting known

In [ ]:
both_exp_data.species.subset('DNMT').require_n_sig(n_sig=1, index='label').heatmap(
    index='label',
    convert_to_log=False,
    columns=['drug', 'stage'],
        figsize=(6, 6),
    cluster_col=False,
    cluster_row=True,
    cluster_by_set=True,
    linewidths=0.01,
    y_tick_labels=True
    
)

plt.savefig("dnmt_proteins.png", dpi=300, bbox_inches='tight')

In [ ]:
exp_data.species.subset('BCL').require_n_sig(n_sig=1, index='label').heatmap(
    index='label',
    convert_to_log=False,
    columns=['drug', 'stage'],
        figsize=(8, 6),
    cluster_col=False,
    cluster_row=True,
    cluster_by_set=True,
    linewidths=0.01,
    y_tick_labels=True
    
)
plt.savefig("bcl_proteins.png", dpi=300, bbox_inches='tight')

### Explore overlaps of samples

In [ ]:
from upsetplot import UpSet, from_contents

In [ ]:
sig_per_sample = dict()
for i in enr_results_up.sample_id.unique():
    terms = enr_results_up.loc[enr_results_up.sample_id == i].sig.term_name
    sig_per_sample[i] = set(terms.values)
sig_per_sample
late_samples = {i:j for i,j in sig_per_sample.items() if 'Late' in i}
late_upset = from_contents(late_samples)
late_upset.rename({'id': 'term'},axis=1, inplace=True)

In [ ]:
proteins_per_sample = dict()
for i,j in zip(exp_data.species.sample_ids, exp_data.species.sig.by_sample):
    proteins_per_sample[i]=j
    
def merge_pair(prefix, samples):
    e = f'{prefix}_Early'
    l = f'{prefix}_Late'
    samples[prefix] = samples[e].union(samples[l])
    del samples[e]
    del samples[l]
    
for i in ['GV', 'G', 'GVD', 'GD']:
    merge_pair(i, proteins_per_sample)
proteins_per_sample['D'] = proteins_per_sample['D_only']
del proteins_per_sample['D_only']

upset_protein = from_contents(proteins_per_sample)
upset_protein.rename({'id': 'term'}, axis=1, inplace=True)
upset_protein.sort_index().head(10)
fig = plt.figure(figsize=(8, 4))
p = UpSet(upset_protein, subset_size='count', show_counts=True, element_size=None, min_subset_size=10).plot(fig=fig)
plt.savefig('upset_plot_global_samples.png', bbox_inches='tight', dpi=300)
plt.savefig('upset_plot_global_samples.pdf', bbox_inches='tight', dpi=300)

In [ ]:

decitabine_samples_protein = {i:j for i,j in proteins_per_sample.items() if 'D' in i or i=='G'}
dec_upset_protein = from_contents(decitabine_samples_protein)
dec_upset_protein.rename({'id': 'term'}, axis=1, inplace=True)
dec_upset_protein.sort_index().head(10)
fig = plt.figure(figsize=(8, 4))
p = UpSet(
    dec_upset_protein, 
    subset_size='count', show_counts=True,
    min_subset_size=10,  
    element_size=30, 
          orientation='horizontal'
).plot(fig=fig)
plt.savefig('upset_plot_dec_samples.png', bbox_inches='tight', dpi=300)
plt.savefig('upset_plot_dec_samples.pdf', bbox_inches='tight', dpi=300)

In [ ]:
from magine.plotting.venn_diagram_maker import create_venn2, create_venn3

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax= fig.add_subplot(121)
create_venn2(exp_data.GD_Early.sig.label_list, exp_data.GD_Late.sig.label_list, 'early', 'late', ax=ax, title='GD')
ax= fig.add_subplot(122)
create_venn2(exp_data.GVD_Early.sig.label_list, exp_data.GVD_Late.sig.label_list, 'early', 'late' , ax=ax, title='GVD')
plt.show()

In [ ]:
gvd = exp_data.GVD_Early.sig.label_list.union(exp_data.GVD_Late.sig.label_list)
gd = exp_data.GD_Early.sig.label_list.union(exp_data.GD_Late.sig.label_list)
gv = exp_data.GV_Early.sig.label_list.union(exp_data.GV_Late.sig.label_list)
g = exp_data.G_Early.sig.label_list.union(exp_data.G_Late.sig.label_list)
d = exp_data.D_only.sig.label_list

In [ ]:
fig = plt.figure(figsize=(4, 4))
ax= fig.add_subplot(111)
create_venn2(
      exp_data.GV_Early.sig.label_list,
    exp_data.GV_Late.sig.label_list,
    
   'GV early', 'GV late', 
    ax=ax
)
plt.savefig('gv_early_vs_late.png', dpi=300, bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(4, 4))
ax= fig.add_subplot(111)
create_venn3(
     d,
    gd,
    gvd,
   'D', 'GD',
    'GVD', 
    ax=ax
)
plt.savefig('decitabine_samples_overlap.png', dpi=300, bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax= fig.add_subplot(111)
create_venn2(
    gvd,
    gv,
    'GVD', 'GV',
    ax=ax
);

In [ ]:
combo_only = gvd.difference(gd).difference(gv).difference(d).difference(g)
combo_run_list = [i.split('_')[0] for i in combo_only]
len(combo_run_list)
plot_gene_list(combo_only, figsize=(8, 12), savename='combo_only', index='label')

In [ ]:

gd_only = gd.difference(gvd).difference(gv).difference(d).difference(g)
gd_only_run_list = [i.split('_')[0] for i in gd_only]
len(gd_only)
plot_gene_list(gd_only, figsize=(8, 12), savename='gd_only', index='label')

In [ ]:
len(gd_only)

In [ ]:
gd_only = gd.difference(gvd).difference(gv).difference(d).difference(g)
gd_only_run_list = [i.split('_')[0] for i in gd_only]

plot_gene_list(gd_only, figsize=(8, 12), savename='gd_only', index='label')

In [ ]:
from magine.enrichment.enrichr import Enrichr

e = Enrichr()
libs = [
    'Reactome_2022',
    'ChEA_2022',
    'ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X',
    'TRRUST_Transcription_Factors_2019', 'CellMarker_2024', 'HuBMAP_ASCTplusB_augmented_2022'
]

In [ ]:
combo_only_enrichment = e.run(combo_only, gene_set_lib=libs)
combo_only_enrichment['sample_id'] = 'combo_only'

In [ ]:
c_only_reactome = combo_only_enrichment.filter_multi(db='Reactome_2022')

In [ ]:
c_only_reactome.filter_multi(p_value=.3).heatmap(
    figsize=(10, 10),
    cluster_row=False,
    # columns=['drug', 'stage'],
    y_tick_labels=True,
    linewidths=0.01
);

In [ ]:
plot_subset(c_only_reactome, 'deubiquitination r-hsa-5688426')

In [ ]:
combo_only_enrichment.filter_multi(p_value=.3).sig.heatmap(
    figsize=(10, 10),
    cluster_row=False,
    # columns=['drug', 'stage'],
    y_tick_labels=True,
    linewidths=0.01
);

In [ ]:
dec_only = gvd.intersection(gd).intersection(d).difference(gv).difference(g)

In [ ]:
run_list_dec_only = [i.split('_')[0] for i in dec_only]

In [ ]:
dec_only_enrichment = e.run(run_list_dec_only, gene_set_lib=libs)
dec_only_enrichment['sample_id'] = 'combo_only'

In [ ]:
d_only_reactome = dec_only_enrichment.filter_multi(db='Reactome_2022')

In [ ]:
d_only_reactome.sig.remove_redundant().heatmap(
    figsize=(4, 6),
    cluster_row=False,
    # columns=['drug', 'stage'],
    y_tick_labels=True,
    linewidths=0.01
);

In [ ]:
plot_gene_list(dec_only, figsize=(8, 12), savename='dec_only', index='label')